# pre processing all current raw data for action id process
    steps:
    1 - devide all files by user (with related information)
    2 - **merge each user's all related files**
    3 - **sort by timestamp**
    4 - decide the certain "action key"(exp. 1:login 2:longoff)
    5 - break the log into action keys (origional using auto-generated non-repeat num ID with parsed log type)
    6 - new line while "logoff" (origional using block_id which not exist here)

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

LDAP_dir = "./r5.2/LDAP"
Processed_dir = "./pre_processed"
result_dir = "./processed_log"

In [2]:
# get LDAP name list for matching user and email

LDAP_files = []
for _, _, file in os.walk(LDAP_dir):
    LDAP_files.append(file)
LDAP_files = np.array(LDAP_files)
LDAP_files = np.squeeze(LDAP_files)

user_file = pd.read_csv(os.path.join(LDAP_dir, LDAP_files[0]))
print(user_file.shape)
print(user_file.head(5))

for file in LDAP_files:
    temp = pd.read_csv(os.path.join(LDAP_dir, file))
    temp = temp[~temp.user_id.isin(user_file.user_id)]
    user_file = user_file.append(temp)
print(user_file.shape)

(1000, 9)
            employee_name  user_id                            email  \
0      Sara Indigo Berger  SIB0825      Sara.Indigo.Berger@dtaa.com   
1       Nerea Ingrid Good  NIG0433       Nerea.Ingrid.Good@dtaa.com   
2     Maggy Maryam Newman  MMN0579     Maggy.Maryam.Newman@dtaa.com   
3      Asher Lucas Daniel  ALD0521      Asher.Lucas.Daniel@dtaa.com   
4  Jael Phyllis Contreras  JPC0464  Jael.Phyllis.Contreras@dtaa.com   

                   role  business_unit             functional_unit  \
0              Salesman              1       5 - SalesAndMarketing   
1  ProductionLineWorker              1           3 - Manufacturing   
2              Salesman              1       5 - SalesAndMarketing   
3         Mathematician              1  2 - ResearchAndEngineering   
4              Salesman              1       5 - SalesAndMarketing   

     department               team                supervisor  
0     2 - Sales  5 - RegionalSales       Regan Kirestin Frye  
1  3 - Assembly 

In [8]:
def show(file):
    print("shape is ",file.shape)

    print("\nindex and top 5 rows")
    print(file.head(5))

In [9]:
user_file = user_file.drop(columns=["employee_name","role","business_unit","functional_unit","department","team","supervisor"])
show(user_file)

shape is  (1000, 2)

index and top 5 rows
   user_id                            email
0  SIB0825      Sara.Indigo.Berger@dtaa.com
1  NIG0433       Nerea.Ingrid.Good@dtaa.com
2  MMN0579     Maggy.Maryam.Newman@dtaa.com
3  ALD0521      Asher.Lucas.Daniel@dtaa.com
4  JPC0464  Jael.Phyllis.Contreras@dtaa.com


In [38]:
def get_email_by_id(df, id):
    return df[(df['user_id'] == id)].email.to_string().split()[1]


In [39]:
# then test we can use the id to query email address
get_email_by_id(user_file, "SIB0825")

'Sara.Indigo.Berger@dtaa.com'

test for one user

In [42]:
test_user_id  = "TDG0203"
file_list = []
for _, _, file in os.walk(Processed_dir+'/'+test_user_id):
    file_list.append(file)
file_list

[['logon_file.csv', 'http_file.csv', 'file_file.csv', 'device_file.csv']]

In [ ]:
test_email = get_email_by_id(user_file, test_user_id)
test_email

In [48]:
test_email_file = Processed_dir+'/'+test_email+"/email_file.csv"
email_file = pd.read_csv(test_email_file)
show(email_file)

shape is  (4659, 2)

index and top 5 rows
                  date                          from
0  01/01/2010 11:18:17  Thomas.Dale.Gilbert@dtaa.com
1  01/01/2010 12:12:10  Thomas.Dale.Gilbert@dtaa.com
2  01/01/2010 13:30:56  Thomas.Dale.Gilbert@dtaa.com
3  01/01/2010 13:41:15  Thomas.Dale.Gilbert@dtaa.com
4  01/01/2010 13:55:07  Thomas.Dale.Gilbert@dtaa.com


In [54]:
email_file = email_file.drop(columns=["from"])

In [55]:
email_file['action'] = "email"

In [56]:
show(email_file)



shape is  (4659, 2)

index and top 5 rows
                  date action
0  01/01/2010 11:18:17  email
1  01/01/2010 12:12:10  email
2  01/01/2010 13:30:56  email
3  01/01/2010 13:41:15  email
4  01/01/2010 13:55:07  email


In [64]:
logon_file = pd.read_csv(Processed_dir+'/'+test_user_id+'/'+file_list[0][0])
http_file = pd.read_csv(Processed_dir+'/'+test_user_id+'/'+file_list[0][1])
file_file = pd.read_csv(Processed_dir+'/'+test_user_id+'/'+file_list[0][2])
device_file = pd.read_csv(Processed_dir+'/'+test_user_id+'/'+file_list[0][3])

In [65]:
show(logon_file)

shape is  (1156, 3)

index and top 5 rows
                  date     user activity
0  01/01/2010 08:04:00  TDG0203    Logon
1  01/01/2010 20:03:00  TDG0203   Logoff
2  01/02/2010 07:53:00  TDG0203    Logon
3  01/02/2010 12:18:06  TDG0203    Logon
4  01/02/2010 20:02:00  TDG0203   Logoff


In [67]:
logon_file = logon_file.drop(columns='user')
show(logon_file)

shape is  (1156, 2)

index and top 5 rows
                  date activity
0  01/01/2010 08:04:00    Logon
1  01/01/2010 20:03:00   Logoff
2  01/02/2010 07:53:00    Logon
3  01/02/2010 12:18:06    Logon
4  01/02/2010 20:02:00   Logoff


In [70]:
logon_file = logon_file.rename(columns={'activity': 'action'})
show(logon_file)

shape is  (1156, 2)

index and top 5 rows
                  date  action
0  01/01/2010 08:04:00   Logon
1  01/01/2010 20:03:00  Logoff
2  01/02/2010 07:53:00   Logon
3  01/02/2010 12:18:06   Logon
4  01/02/2010 20:02:00  Logoff


In [71]:
http_file = http_file.drop(columns='user')
show(http_file)

shape is  (21422, 2)

index and top 5 rows
                  date action
0  01/01/2010 08:31:04   http
1  01/01/2010 08:37:25   http
2  01/01/2010 08:43:17   http
3  01/01/2010 09:42:31   http
4  01/01/2010 09:51:00   http


In [72]:
file_file = file_file.drop(columns='user')
show(file_file)

shape is  (1488, 2)

index and top 5 rows
                  date action
0  01/02/2010 16:18:15   file
1  01/02/2010 16:40:19   file
2  01/02/2010 17:14:00   file
3  01/04/2010 09:19:32   file
4  01/04/2010 09:51:23   file


In [73]:
device_file = device_file.drop(columns='user')
show(device_file)

shape is  (818, 2)

index and top 5 rows
                  date    activity
0  01/02/2010 15:44:35     Connect
1  01/02/2010 17:53:47  Disconnect
2  01/04/2010 09:18:20     Connect
3  01/04/2010 12:35:28  Disconnect
4  01/05/2010 17:25:52     Connect


In [74]:
device_file = device_file.rename(columns={'activity': 'action'})
show(device_file)

shape is  (818, 2)

index and top 5 rows
                  date      action
0  01/02/2010 15:44:35     Connect
1  01/02/2010 17:53:47  Disconnect
2  01/04/2010 09:18:20     Connect
3  01/04/2010 12:35:28  Disconnect
4  01/05/2010 17:25:52     Connect


In [77]:
final_df = logon_file.append(http_file).append(file_file).append(device_file).append(email_file)
show(final_df)

shape is  (29543, 2)

index and top 5 rows
                  date  action
0  01/01/2010 08:04:00   Logon
1  01/01/2010 20:03:00  Logoff
2  01/02/2010 07:53:00   Logon
3  01/02/2010 12:18:06   Logon
4  01/02/2010 20:02:00  Logoff


In [78]:
final_df.to_csv("./test_final_raw.csv")
